In [1]:
# -*- coding: utf-8 -*-
'''
TopQuant-简称TQ极宽智能量化回溯分析系统，培训课件-配套教学python课件程序

Top极宽量化(原zw量化)，Python量化第一品牌 
by Top极宽·量化开源团队 2017.10.1 首发

网站： www.TopQuant.vip      www.ziwang.com
QQ群: Top极宽量化1群，124134140
      Top极宽量化2群，650924099
      Top极宽量化3群，450853713
  
'''

import os
import pandas as pd
import numpy as np
import tflearn
import tensorflow as tf
#
import ztools as zt
import ztools_data as zdat

#
import zsys
import ztools_tq as ztq

#------------------

#1
print('\n#1,set.sys')
pd.set_option('display.width', 450)    
pd.set_option('display.float_format', zt.xfloat3)    
rlog='ailib/log_tmp'
if os.path.exists(rlog):tf.gfile.DeleteRecursively(rlog)



#2
print('\n#2,读取数据')
#fss='/zdat/cn/xday/000001.csv'
fss='data/inx_000001.csv'
df=pd.read_csv(fss,index_col=0)
df=df.sort_index()

#3
print('\n#3,整理数据')
df['xopen']=df['open'].shift(-1)
df['xclose']=df['close'].shift(-1)
df['kclose']=df['xclose']/df['xopen']*100
df['ktype']=df['kclose'].apply(zt.iff3type,d0=99.5,d9=101.5,v3=3,v2=2,v1=1)
df['ktype']=df['ktype']-1
#


#4
print('\n#4,设置训练数据')
n9=len(df.index)
df1=df.head(2000)
df2=df.tail(n9-2000)
#
X=df1[zsys.ohlcLst].values
Y=df1['ktype'].values
y1s=pd.get_dummies(Y)
y1=y1s.values
#

#5
print('\n#5,构建线性回归神经网络模型')

# 选择OHLC字段作为输入
net = tflearn.input_data(shape=[None, 4])
#net = tflearn.fully_connected(net, 40)
# 全连接层，每层40个节点
net = tflearn.fully_connected(net, 40)

# 输出3类
net = tflearn.fully_connected(net, 3, activation='softmax')
net = tflearn.regression(net)
#
m = tflearn.DNN(net,tensorboard_dir=rlog)
#
#6
print('\n#6,开始训练模型')
m.fit(X, y1, n_epoch=100, show_metric=True)

#7
print('\n#7,根据模型，进行预测')
X2=df2[zsys.ohlcLst].values
Y2=m.predict(X2)
#
y2v=map(np.argmax,Y2)
ds2y=zdat.ds4x(y2v,df2.index)
df2['ktype2']= ds2y
#
print(df2.tail())
df2.to_csv('tmp/df2.csv')
#
#8
print('\n#8,计算预测结果')
df5=pd.DataFrame()
df5['y_test']=df2['ktype']
df5['y_pred']=df2['ktype2']
acc,df5x=ztq.ai_acc_xed2x(df5['y_test'],df5['y_pred'],ky0=0.5)
#
print('\nacc,',acc)
print(df5.tail())
df5.to_csv('tmp/df5.csv')

#9
print('\n#9,ok')




Training Step: 3199  | total loss: 0.96672 | time: 0.114s
| Adam | epoch: 100 | loss: 0.96672 - acc: 0.4879 -- iter: 1984/2000
Training Step: 3200  | total loss: 0.96802 | time: 0.116s
| Adam | epoch: 100 | loss: 0.96802 - acc: 0.5027 -- iter: 2000/2000
--

#7,根据模型，进行预测
               open     high    close      low       volume        amount    xopen   xclose  kclose  ktype  ktype2
date                                                                                                              
2017-04-26 3132.918 3152.953 3140.847 3131.418  16987810700  197112873017 3131.350 3152.187 100.665      1       1
2017-04-27 3131.350 3155.003 3152.187 3097.333  21179307300  235748319355 3144.022 3154.658 100.338      1       1
2017-04-28 3144.022 3154.727 3154.658 3136.578  16288989900  183195769806 3147.228 3143.712  99.888      1       1
2017-05-02 3147.228 3154.781 3143.712 3136.539  15422296200  176389916688 3138.307 3135.346  99.906      1       1
2017-05-03 3138.307 3148.286 3135.346 3

/Users/bluegrey/py3env/lib/python3.6/site-packages/ipykernel_launcher.py:89: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
